In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
mnist = input_data.read_data_sets("data/fashion", one_hot=True)

learning_rate = 0.001

# 网络参数
n_input = 784 
n_classes = 10 
dropout = 0.75 


with tf.name_scope('input_data') as scope:
    x = tf.placeholder(tf.float32, [None, n_input] ,name='input')
    y = tf.placeholder(tf.float32, [None, n_classes], name='label')
    keep_prob = tf.placeholder(tf.float32, shape=(), name='drop_out') 


def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],padding='SAME')


# 创建模型
def conv_net(x,dropout):
    with tf.name_scope('model') as scope:
        x = tf.reshape(x, shape=[-1, 28, 28, 1])
        # 卷积层
        # Max Pooling (down-sampling)
        with tf.name_scope('layer1') as scope:
            W_conv1 = tf.Variable(tf.random_normal(shape=[5,5,1,32]), name='weight')
            b_conv1 = tf.Variable(tf.random_normal(shape=[1,32]), name='bias')
            convOne = tf.nn.conv2d(x, W_conv1, strides=[1,1,1,1], padding="SAME")
            reluOne = tf.nn.relu(convOne + b_conv1)
            conv1 = tf.nn.max_pool(reluOne, ksize=[1,2,2,1],strides=[1,2,2,1],padding="SAME")

        # 卷积层
        with tf.name_scope('layer2') as scope:
            W_conv2 = tf.Variable(tf.random_normal(shape=[5,5,32,64]), name='weight')
            b_conv2 = tf.Variable(tf.random_normal( shape=[1,64]), name='bias')
            convTwo = tf.nn.conv2d(conv1, W_conv2, strides=[1,1,1,1], padding="SAME")
            reluTwo = tf.nn.relu(convTwo + b_conv2)
            conv2 = tf.nn.max_pool(reluTwo, ksize=[1,2,2,1], strides=[1,2,2,1],padding="SAME")

        # 全连接层
        # Reshape conv2 适合全连接层的输入输出
        with tf.name_scope('full_connect') as scope:
            W_full = tf.Variable(tf.random_normal(shape=[7 * 7 * 64, 1024]), name='weight')
            b_full = tf.Variable(tf.random_normal(shape=[1, 1024]), 'bias')
            input_flat=tf.reshape(conv2, shape=[-1, 7 * 7 * 64])
            fc1 = tf.nn.relu(tf.matmul(input_flat, W_full) + b_full)

        with tf.name_scope('soft_max') as scope:
            drop_out = tf.nn.dropout(fc1,keep_prob)

        # 类预测输出
            W_softmax = tf.Variable(tf.truncated_normal(shape=[1024, 10]), name='weight')
            b_softmax = tf.Variable(tf.truncated_normal(shape=[1,10]), name='bias')
            y_predict = tf.matmul(drop_out, W_softmax) + b_softmax

        return y_predict
    # 构建模型

pred = conv_net(x, keep_prob)

with tf.name_scope('result') as scope:
    # 定义损失和优化器
    cross_entropy_cnn = -y * tf.nn.log_softmax(pred)
    cost = tf.reduce_sum(cross_entropy_cnn, name='cost') ### 目标函数有问题

    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

    # 评估模型
    correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')

cost_summary = tf.summary.scalar(cost.op.name, cost)
accuracy_summary = tf.summary.scalar(accuracy.op.name, accuracy)

# 初始化变量
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(tf.initialize_all_variables())

summary_op = tf.summary.merge([cost_summary, accuracy_summary])
summary_writer = tf.summary.FileWriter('/home/artemis/graphs',sess.graph)

for i in range(500):
    batch = mnist.train.next_batch(128)

    if i% 100 == 0:
        train_accuracy = accuracy.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0},session=sess)
        print ("step "+ str(i) +", training accuracy :"+ str(train_accuracy))
        cross_entropy_val = cross_entropy_cnn.eval(feed_dict={x:batch[0], y:batch[1], keep_prob:1.0},session=sess)

        summary_str = sess.run(summary_op, feed_dict={x:batch[0], y:batch[1], keep_prob:(1.0)})
        summary_writer.add_summary(summary_str, i)

    sess.run(optimizer, feed_dict={x:batch[0], y:batch[1], keep_prob:0.75})
print("test accuracy :" + str(accuracy.eval(feed_dict={x:mnist.test.images, y:mnist.test.labels, keep_prob:1.0},session=sess)))
sess.close()
summary_writer.close()  ##程序运行结束后关闭文件并刷新到硬盘

Extracting data/fashion/train-images-idx3-ubyte.gz
Extracting data/fashion/train-labels-idx1-ubyte.gz
Extracting data/fashion/t10k-images-idx3-ubyte.gz
Extracting data/fashion/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.global_variables_initializer` instead.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
step 0, training accuracy :0.09375
step 100, training accuracy :0.804688
step 200, training accuracy :0.796875
step 300, training accuracy :0.8125
step 400, training accuracy :0.867188
test accuracy :0.8271
